In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

from matplotlib import rc
rc('font', family='AppleGothic') #Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

from scipy.stats import boxcox

In [17]:
def preprocess_data(data):

    data.drop('매출액', axis=1, inplace=True)
    data.drop('매출액점유율', axis=1, inplace=True)
    data.drop('누적매출액', axis=1, inplace=True)
    data.drop('누적관객수', axis=1, inplace=True)
    data.drop('상영횟수', axis=1, inplace=True)
    data.drop('국적', axis=1, inplace=True) 
    data.drop('제작사', axis=1, inplace=True)
    data.drop('스크린수', axis=1, inplace=True)

    data.rename(columns={'영화명': 'title'}, inplace=True)
    data.rename(columns={'개봉일': 'release_date'}, inplace=True)
    data.rename(columns={'관객수': 'box_off_num'}, inplace=True)
    data.rename(columns={'대표국적': 'country'}, inplace=True)
    data.rename(columns={'등급': 'screening_rat'}, inplace=True)
    data.rename(columns={'장르': 'genre'}, inplace=True)
    data.rename(columns={'배급사': 'distributor'}, inplace=True)
    data.rename(columns={'감독': 'director'}, inplace=True)
    data.rename(columns={'배우': 'actors'}, inplace=True)

    data['title'] = data['title'].str.replace(' ', '')
    data['country'] = data['country'].str.replace(' ', '')
    data['screening_rat'] = data['screening_rat'].str.replace(' ', '')
    data['genre'] = data['genre'].str.replace(' ', '')
    data['distributor'] = data['distributor'].str.replace(' ', '')
    data['director'] = data['director'].str.replace(' ', '')
    data['actors'] = data['actors'].str.replace(' ', '')
    
    #title
    data = data.dropna(subset=['director', 'actors'])

    data = data.sort_values('box_off_num', ascending=False).drop_duplicates('title')
    
    data = data.dropna(subset=['title'])

    data = data[~data['title'].str.contains('감독판')]

    #release_date
    data = data[(data['release_date'].dt.year >= 2023) & (data['release_date'].dt.year < 2024)]
    
    #data = data[(data['release_date'].dt.year >= 2024)] #2024년 데이터 전처리 할 시

    data['release_date'] = pd.to_datetime(data['release_date'])

    def get_season(date):
        if date.month in [3, 4, 5]:
            return 'spring'
        elif date.month in [6, 7, 8]:
            return 'summer'
        elif date.month in [9, 10, 11]:
            return 'autumn'
        else:
            return 'winter'

    data['season'] = data['release_date'].apply(get_season)
    
    #country
    def group_country(country):
        if country in ['미국', '캐나다','멕시코']:
            return '북중미'
        elif country == '한국':
            return '한국'
        elif country == '일본':
            return '일본'
        elif country in ['중국', '홍콩', '대만']:
            return '중화권'
        elif country in ['프랑스', '영국', '독일','스페인','이탈리아','덴마크', '스웨덴', '네덜란드', '벨기에', 
                        '아일랜드', '노르웨이', ',아르헨티나', ',핀란드', '오스트리아', '아이슬란드', '헝가리']:
            return '유럽'
        else:
            return '기타'
    
    data['country'] = data['country'].apply(group_country)
    
    #genre

    data['genre'].fillna('기타', inplace=True)
    data['genre'] = data['genre'].str.split(',').str[0]

    data.loc[data['genre'] == '가족', 'genre'] = '드라마'
    data.loc[data['genre'] == '공연', 'genre'] = '뮤지컬'
    data.loc[data['genre'] == '서부극(웨스턴)', 'genre'] = '액션'
    data.loc[data['genre'] == '전쟁', 'genre'] = '사극'
    data.loc[data['genre'] == '스릴러', 'genre'] = '공포(호러)'

    data['genre'].replace('멜로/로맨스', '로맨스', inplace=True)
    data['genre'].replace('공포(호러)', '공포', inplace=True)
    data['genre'].replace('뮤지컬', '예술', inplace=True)
    data['genre'].replace('사극', '역사/전쟁', inplace=True)

    data = data[data['genre'] != '성인물(에로)']
    data = data[data['genre'] != '기타']
    
    #최종발표 이후 장르 전처리 방식 바꿀 시
    
    # data['genre'].fillna('기타', inplace=True)
    # data = data[~data['genre'].str.contains('성인물|기타')]
    
    # all_genres = set()
    # for genres in data['genre']:
    #     all_genres.update(genres.split(','))

    # for genre in all_genres:
    #     data['genre_' + genre] = data['genre'].apply(lambda x: 1 if genre in x.split(',') else 0)

    #screening_rat
    data['screening_rat'].fillna('기타', inplace=True)

    data['screening_rat'] = data['screening_rat'].str.split(',').str[0]

    def get_dis(x) :
        if '12세이상관람가' in x or '12세관람가' in x or '12세미만인자는관람할수없는등급' in x or '국민학생관람불가' in x :
            return '만12세이상관람가'
        elif '15세이상관람가' in x or '15세관람가' in x :
            return '만15세이상관람가'
        elif '18' in x or '청소년관람불가' in x or '15세미만인자는관람할수없는등급' in x or '고등학생이상관람가' in x or '연소자관람불가' in x or '18세관람가' in x or '미성년자관람불가' in x:
            return '만18세이상관람가'
        elif '전체관람가' in x or '모든관람객이관람할수있는등급' in x or '연소자관람가' in x:
            return '전체관람가'
        elif '' in x or '미정' in x or '기타' in x:
            return '기타'
        else :
            return '기타'
    
    data['screening_rat'] = data.screening_rat.apply(get_dis)

    data = data[data['screening_rat'] != '기타']

    #distributor
    data = data.dropna(subset=['distributor'])
    data['distributor'] = data['distributor'].str.split(',').str[0].str.strip()
    data['distributor'] = data.distributor.str.replace("(주)", '')
    data['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in data.distributor]

    def get_dis(x) :
        if 'CJENM' in x or '씨제이씨지브이CJCGV' in x or '씨제이이엔엠' in x or 'CGV아트하우스' in x:
            return 'CJENM'
        elif '롯데컬처웍스롯데엔터테인먼트' in x or '롯데쇼핑롯데엔터테인먼트' in x or '롯데쇼핑롯데시네마' in x: 
            return '롯데엔터테인먼트'
        elif '쇼박스' in x: 
            return '쇼박스'
        elif '넥스트엔터테인먼트월드NEW' in x or '콘텐츠판다' in x: 
            return 'NEW'
        elif '플러스엠' in x or '플러스엠 엔터테인먼트' in x or '메가박스' in x: 
            return '플러스엠'
        elif '에스케이플래닛' in x or '에스케이텔레콤' in x: 
            return 'SK'
        
        elif '월트디즈니컴퍼니코리아유한책임회사' in x or '월트디즈니컴퍼니코리아' in x: 
            return 'Disney'
        elif '소니픽쳐스릴리징월트디즈니스튜디오스코리아' in x or '소니픽쳐스엔터테인먼트코리아주식회사극장배급지점' in x or '한국소니픽쳐스릴리징브에나비스타영화' in x:
            return '소니'
        elif '유니버설픽쳐스인터내셔널코리아유' in x :
            return 'UPI'
        
        else :
            return x
        
    data['distributor'] = data.distributor.apply(get_dis)

    dist_power = pd.read_excel('/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/calculated_power_data/dist_power.xlsx', index_col=0)
    dist_power = dist_power.reset_index()
    data = pd.merge(data, dist_power[['distributor', 'dist_power', 'dist_power_weighted']], on='distributor', how='left')
    
    avg_dist_power = dist_power['dist_power'].mean()
    data['dist_power'].fillna(avg_dist_power, inplace=True)
    avg_dist_power_weighted = dist_power['dist_power_weighted'].mean()
    data['dist_power_weighted'].fillna(avg_dist_power_weighted, inplace=True)

    #director
    data['director'] = data['director'].str.split(',').str[0]
    
    dir_power = pd.read_excel('/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/calculated_power_data/dir_power.xlsx', index_col=0)
    dir_power = dir_power.reset_index()
    data = pd.merge(data, dir_power[['director', 'dir_power', 'dir_power_weighted']], on='director', how='left')
    
    avg_dir_power = dir_power['dir_power'].mean()
    data['dir_power'].fillna(avg_dir_power, inplace=True)
    avg_dir_power_weighted = dir_power['dir_power_weighted'].mean()
    data['dir_power_weighted'].fillna(avg_dir_power_weighted, inplace=True)

    #actor
    def keep_top_3_actors(actors_str):
        if isinstance(actors_str, str):
            actors_list = actors_str.split(',')
            top_3_actors = ', '.join(actors_list[:3])
            return top_3_actors
        else:
            return ''

    data['main_actors'] = data['actors'].apply(keep_top_3_actors)

    act_power = pd.read_excel('/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/calculated_power_data/act_power.xlsx')

    def calculate_actor_power(row):
        if isinstance(row['main_actors'], str):
            actors = row['main_actors'].split(', ')
            avg_act_power = []
            avg_act_power_weighted = []
            for actor in actors:
                actor_power_row = act_power[act_power['unique_actors'] == actor.strip()]
                if not actor_power_row.empty:
                    avg_act_power.append(actor_power_row['act_power'].values[0])
                    avg_act_power_weighted.append(actor_power_row['act_power_weighted'].values[0])
            avg_act_power_mean = sum(avg_act_power) / len(avg_act_power) if avg_act_power else None
            avg_act_power_weighted_mean = sum(avg_act_power_weighted) / len(avg_act_power_weighted) if avg_act_power_weighted else None
            return avg_act_power_mean, avg_act_power_weighted_mean
        else:
            return None, None

    data['act_power'], data['act_power_weighted'] = zip(*data.apply(calculate_actor_power, axis=1))

    avg_act_power_mean = act_power['act_power'].mean()
    avg_act_power_weighted_mean = act_power['act_power_weighted'].mean()

    data['act_power'].fillna(avg_act_power_mean, inplace=True)
    data['act_power_weighted'].fillna(avg_act_power_weighted_mean, inplace=True)

    #로그변환
    data['dist_power_weighted_log'] = np.log1p(data['dist_power_weighted'])
    data['dir_power_weighted_log'] = np.log1p(data['dir_power_weighted'])
    data['act_power_weighted_log'] = np.log1p(data['act_power_weighted'])
    
    #data['box_off_num'] = np.log1p(data['box_off_num'])

    #series
    data['series'].fillna(0, inplace=True)
    
    #dummy encoding
    data = pd.get_dummies(data, columns=['season', 'country', 'screening_rat', 'genre'], drop_first=True, dtype=int)
    #data = pd.get_dummies(data, columns=['season', 'country', 'screening_rat'], drop_first=True, dtype=int) #최종발표 이후 장르 전처리 방식 바꿀 시
    
    #불필요컬럼제거
    data = data.drop(columns=['release_date','distributor','director', 'actors','main_actors', 'dist_power', 'dir_power', 'act_power','dist_power_weighted', 'dir_power_weighted', 'act_power_weighted'])
    
    #data = data.drop(columns=['release_date','distributor','genre','director', 'actors','main_actors', 'dist_power', 'dir_power', 'act_power','dist_power_weighted', 'dir_power_weighted', 'act_power_weighted']) #최종발표 이후 장르 전처리 방식 바꿀 시
    
    return data


In [18]:
raw_data_2023 = pd.read_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2023년_data/2023_raw data.xlsx")
preprocessed_data_2023 = preprocess_data(raw_data_2023)

data_title = preprocessed_data_2023[['title']]
data_title.to_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2023년_data/2023_data_title.xlsx", index=False)

data_answer = preprocessed_data_2023[['title','box_off_num']]
data_answer.rename(columns={'box_off_num': 'actual_box_off_num'}, inplace=True)
data_answer.to_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2023년_data/2023_data_answer.xlsx", index=False)

preprocessed_data_2023 = preprocessed_data_2023.drop(columns=['title', 'box_off_num'])
preprocessed_data_2023.to_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2023년_data/2023_preprocessed_data.xlsx", index=False)

preprocessed_data_2023.head()

,series,genre_공포(호러),genre_서부극(웨스턴),genre_멜로/로맨스,genre_액션,genre_어드벤처,genre_SF,genre_스릴러,genre_범죄,genre_사극,...,season_summer,season_winter,country_북중미,country_유럽,country_일본,country_중화권,country_한국,screening_rat_만15세이상관람가,screening_rat_만18세이상관람가,screening_rat_전체관람가
0,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1.0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
2,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
4,1.0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
raw_data_2024 = pd.read_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2024년_data/2024_raw data.xlsx")
preprocessed_data_2024 = preprocess_data(raw_data_2024)

data_title = preprocessed_data_2024[['title']]
data_title.to_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2024년_data/2024_data_title.xlsx", index=False)

data_answer = preprocessed_data_2024[['title','box_off_num']]
data_answer.rename(columns={'box_off_num': 'actual_box_off_num'}, inplace=True)
data_answer.to_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2024년_data/2024_data_answer.xlsx", index=False)

preprocessed_data_2024 = preprocessed_data_2024.drop(columns=['title', 'box_off_num'])
preprocessed_data_2024.to_excel("/Users/seojeongmin/Desktop/3-1/데이터사이언스응용/TeamProject/data/2024년_data/2024_preprocessed_data.xlsx", index=False)

preprocessed_data_2024.head()